In [3]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [4]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET

def load_data(image_folder, annotation_folder):
    image_files = os.listdir(image_folder)
    images = []
    annotations = []
    
    for img_file in image_files:
        img_path = os.path.join(image_folder, img_file)
        annotation_path = os.path.join(annotation_folder, img_file.replace('.jpg', '.xml'))
        
        # Load image
        img = cv2.imread(img_path)
        # Resize image to 224x224
        img = cv2.resize(img, (224, 224))  
        images.append(img)
        
        # Load XML annotation
        if os.path.exists(annotation_path):
            # Check if annotation file exists  
            tree = ET.parse(annotation_path)
            root = tree.getroot()
            
            # Extract polygon points
            points = []
            for line in root.findall('.//line'):
                for pt in line.findall('point'):
                    x = int(pt.attrib['x'])
                    y = int(pt.attrib['y'])
                    points.append((x, y))
            annotations.append(points)
        else:
            print(f"No annotation found for {img_file}. Skipping.")
    
    return np.array(images), annotations

# Load data
image_folder = 'Shapes'
annotation_folder = 'Annonated_XML'
images, annotations = load_data(image_folder, annotation_folder)


In [5]:
images[0]

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [6]:
annotations

[[(122, 132),
  (121, 133),
  (119, 133),
  (119, 135),
  (118, 136),
  (118, 301),
  (119, 302),
  (119, 304),
  (121, 304),
  (122, 305),
  (287, 305),
  (288, 304),
  (290, 304),
  (290, 302),
  (291, 301),
  (291, 136),
  (290, 135),
  (290, 133),
  (288, 133),
  (287, 132)],
 [(106, 74),
  (106, 76),
  (105, 77),
  (105, 224),
  (106, 225),
  (106, 227),
  (109, 227),
  (110, 226),
  (111, 226),
  (112, 225),
  (113, 225),
  (115, 223),
  (116, 223),
  (117, 222),
  (118, 222),
  (119, 221),
  (120, 221),
  (122, 219),
  (123, 219),
  (124, 218),
  (125, 218),
  (126, 217),
  (127, 217),
  (129, 215),
  (130, 215),
  (131, 214),
  (132, 214),
  (133, 213),
  (134, 213),
  (136, 211),
  (137, 211),
  (138, 210),
  (139, 210),
  (140, 209),
  (141, 209),
  (143, 207),
  (144, 207),
  (145, 206),
  (146, 206),
  (147, 205),
  (148, 205),
  (150, 203),
  (151, 203),
  (152, 202),
  (153, 202),
  (154, 201),
  (155, 201),
  (157, 199),
  (158, 199),
  (159, 198),
  (160, 198),
  (161, 

In [7]:
# Convert annotations to binary edge images
def annotations_to_edges(annotations, image_shape):
    edge_images = []
    for annotation in annotations:
        edge_img = np.zeros(image_shape[:2], dtype=np.uint8)
        for i in range(len(annotation)):
            cv2.line(edge_img, annotation[i], annotation[(i+1)%len(annotation)], 255, 1)
        edge_images.append(edge_img)
    return np.array(edge_images)

edge_images = annotations_to_edges(annotations, images[0].shape)


# Split data into training and validation sets
from sklearn.model_selection import train_test_split
train_images, val_images, train_edges, val_edges = train_test_split(images, edge_images, test_size=0.2, random_state=42)

# Normalize images
train_images = train_images.astype('float32') / 255.0
val_images = val_images.astype('float32') / 255.0
train_edges = train_edges.astype('float32') / 255.0
val_edges = val_edges.astype('float32') / 255.0

In [8]:
from tensorflow.keras import models, layers

def build_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),  # Added dropout layer for regularization
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),  # Added another dropout layer
        layers.Dense(input_shape[0] * input_shape[1], activation='sigmoid'),  
        layers.Reshape((input_shape[0], input_shape[1]))
    ])
    return model

input_shape = (224, 224, 3)
model = build_model(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



C:\Users\RPA20\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50176)          │     6,472,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 224, 224)       │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,641,664 (67.30 MB)

 Trainable params: 17,641,664 (67.30 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# 3. Training

model.fit(train_images, train_edges, epochs=20, batch_size=32, validation_data=(val_images, val_edges))
model.save("model_attempt4.h5")

Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 51s 819ms/step - accuracy: 0.0882 - loss: 0.2908 - val_accuracy: 0.6158 - val_loss: 0.0826
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 30s 801ms/step - accuracy: 0.5784 - loss: 0.0812 - val_accuracy: 0.6162 - val_loss: 0.0826
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 31s 825ms/step - accuracy: 0.5910 - loss: 0.0783 - val_accuracy: 0.6162 - val_loss: 0.0826
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 31s 809ms/step - accuracy: 0.5908 - loss: 0.0800 - val_accuracy: 0.6162 - val_loss: 0.0826
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 30s 787ms/step - accuracy: 0.6002 - loss: 0.0793 - val_accuracy: 0.6139 - val_loss: 0.0826
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 30s 778ms/step - accuracy: 0.5969 - loss: 0.0794 - val_accuracy: 0.6162 - val_loss: 0.0826
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 29s 771ms/step - accuracy: 0.5887 - loss: 0.0805 - val_accuracy: 0.6162 - val_loss: 0.0826
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 30s 788ms/step - accuracy: 0.5930 - loss: 0.0798 - val_accu

In [12]:
# 4. Inference

def annotate_edge(image_path, model):
    # Load test image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img_norm = img.astype('float32') / 255.0
    img_norm = np.expand_dims(img_norm, axis=0)
    
    # Predict edge map
    edge_map = model.predict(img_norm)[0]
    edge_map = (edge_map > 0.5).astype(np.uint8) * 255
    
    # Convert edge map to XML annotation
    edge_points = np.column_stack(np.where(edge_map > 0))
    
    root = ET.Element("annotation")
    filename = ET.SubElement(root, "filename")
    filename.text = os.path.basename(image_path)
    
    obj = ET.SubElement(root, "object")
    name = ET.SubElement(obj, "name")
    name.text = "shape"
    
    polygon = ET.SubElement(obj, "polygon")
    for point in edge_points:
        pt = ET.SubElement(polygon, "pt")
        x = ET.SubElement(pt, "x")
        x.text = str(point[1])
        y = ET.SubElement(pt, "y")
        y.text = str(point[0])
    
    tree = ET.ElementTree(root)
    xml_path = image_path.replace('.jpg', '_annotated.xml')
    tree.write(xml_path)
    
    # Save annotated image
    contours, _ = cv2.findContours(edge_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
    annotated_img_path = image_path.replace('.jpg', '_annotated.jpg')
    cv2.imwrite(annotated_img_path, img)

# Test annotation on a sample image
test_image_path = 'test_img.jpg'
annotate_edge(test_image_path, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
